In [83]:
import tweepy
import nltk
import pandas as pd
import emoji

In [84]:
# Get Bearer Token
filepath = '../api.key.json'
keys = pd.read_json(filepath, typ='series')
auth = tweepy.OAuth2BearerHandler(keys['BEARER_TOKEN'])
api = tweepy.API(auth)

In [85]:
# iterate over multiple pages of results
max_items = 500
emotions = ['joy', 'fear', 'sadness', 'anger']

dataset = []
for status in tweepy.Cursor(api.search_tweets, "#happiness",count=100).items(max_items):
    json_str = status._json
    timestamp = pd.to_datetime(json_str['created_at']).strftime('%Y-%m-%d %H:%M:%S')
    text, hashtags, symbols = json_str['text'], json_str['entities']['hashtags'], json_str['entities']['symbols']
    dataset.append((timestamp, text, hashtags, symbols))

df = pd.DataFrame(dataset, columns=['CreatedAt', 'Text', 'Hashtags','Symbols'])

print(df.shape)

(500, 4)


In [86]:
from nltk.tokenize import TweetTokenizer
from collections import Counter

# tokenizer = TreebankWordTokenizer()
tokenizer = TweetTokenizer(strip_handles=True)
tokens = df['Text'].apply(tokenizer.tokenize)

counter = Counter([token for token_lists in tokens.values for token in token_lists])
counter.most_common(n=20)